In [29]:
import pyPI as pi
import numpy as np
from scipy.linalg import eig
import sympy as sp
from scipy.optimize import minimize

In [30]:
chi = np.pi/3
alpha = np.pi/4

# Vectores de entrada y salida
E_is = pi.polarization_basis_set('L')
S = pi.jones_matrix(chi, alpha)
E_s = S @ E_is
S

array([[0.75-0.4330127j, 0.25+0.4330127j],
       [0.25+0.4330127j, 0.75-0.4330127j]])

In [31]:
# Ejemplo de uso con vectores específicos:
x_1 = E_is
x_2 = pi.polarization_basis_set('H')

b_1 = E_s
b_2 = S @ x_2

x_vectors = [x_1, x_2]
b_vectors = [b_1, b_2]

A, alpha_calculated = pi.determine_A(x_vectors, b_vectors, p=True)

Determined matrix A:
[[0.75-0.4330127j 0.25+0.4330127j]
 [0.25+0.4330127j 0.75-0.4330127j]] 

α (dominant direction): 0.2500π



In [32]:
# Matriz por minimizacion
optimal_delta_chi, optimal_alpha = pi.birefringence_by_minimization(E_is, E_s, p = True)

χ: 0.3333π
α: 0.2500π


In [33]:
print(f"α original: {alpha / np.pi:.4f}π\n")
print(f"α calculated: {alpha_calculated / np.pi:.4f}π\n")
print(f"α by minimization: {optimal_alpha / np.pi:.4f}π\n")

α original: 0.2500π

α calculated: 0.2500π

α by minimization: 0.2500π



In [34]:
chi_calculated = pi.calculate_chi(alpha, E_is, E_s,p = True)

Calculated value of Δχ: 0.3333π


In [35]:
print(f"Δχ original: {chi / np.pi:.4f}π\n")
print(f"Δχ calculated: {chi_calculated / np.pi:.4f}π\n")
print(f"Δχ by minimization: {optimal_delta_chi / np.pi:.4f}π\n")

Δχ original: 0.3333π

Δχ calculated: 0.3333π

Δχ by minimization: 0.3333π

